<a href="https://colab.research.google.com/github/MiguelAmorim-587/ZenPrice/blob/main/Data_Extraction_and_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data extration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Drive path
%cd "/content/drive/MyDrive/Tese"

Mounted at /content/drive
/content/drive/MyDrive/Tese


In [ ]:
!nvidia-smi

Thu Oct 27 01:58:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Base libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10, 5)

# Convert currencies
!pip install currencyconverter
from currency_converter import CurrencyConverter

# Conver strings with commas as thousand separators
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') 

# Convert datatimes
from datetime import datetime

# Read zips
from zipfile import ZipFile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 557 kB 14.7 MB/s 


In [ ]:
def zip_2_dataframe(zip_name, file_name, filetype):
  zip_file = ZipFile(zip_name, 'r')
  data_file = zip_file.open(file_name)

  if filetype == "csv":
    df = pd.read_csv(data_file)
  if filetype == "json":
    df = pd.read_json(data_file, lines=True)

  return df

In [ ]:
zip_name = "catalog_sample.zip"
file_name = "catalog_sample.json"
filetype = "json"
df_og = zip_2_dataframe(zip_name, file_name, filetype)

## Data Cleaning

In [ ]:
df = df_og.copy()
df_og

,product,country,company,price,timestamp,offer_type
0,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",20200722,unlocked
1,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",20200721,unlocked
2,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",20200720,unlocked
3,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",20200719,unlocked
4,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",20200718,unlocked
...,...,...,...,...,...,...
602051,huawei Y9S 128GB,CL,claro,"$229,990",20200621,unlocked
602052,huawei Y9S 128GB,CO,Claro,"$1,168,900",20200621,unlocked
602053,huawei Y9S 128GB,CO,Tigo,"$1,079,900",20200621,unlocked
602054,huawei Y9S 128GB,PE,Movistar,"S/1,519",20200620,postpaid_portability




*   Convert elements of timestamp column to dates

In [ ]:
def str_2_date(str_date, date_format):
  return datetime.strptime(str(str_date), date_format)

df["timestamp"] = df_og["timestamp"].apply(str_2_date, args = ('%Y%m%d',))

*   List of different currencies

There are three different types of currencies:

- Euros
- Dollars
- Sol (Peru's currency)

**Note:** To compare prices of different countries it may be useful to convert all currencies to a base currency (e.g. euro). However, there are fluctuations in the value of each currency, and so it may be better to do the analysis in each currency, but this should be further analysed.

In [ ]:
df["price"].str[0].unique()

array(['$', '€', 'S'], dtype=object)

To compare prices across different countries, a new column will be created with all prices in Euros. A column with the prices withouth commas, dots and currency signs wil also be created.

The value of each currency as stated by the Europen Central Bank can be found in this link: https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html).

 The Sol (Peru) currency is not present in here.

In [ ]:
def replace_last_occurrence(str, old_char, new_char):

  # Replace the last ocurrence of a char in a string by another char

  last_index = str.rfind(old_char)
  if last_index == -1:
    return str
  else:
    return str[:last_index] + new_char + str[last_index+1:]


def clean_prices(price):

  # Remove commas, dots and currency signs from the prices

  # Sol peruano (doesn't have decimal cases - thousands are separated with commas)
  if price[0] == "S":
    price = price.replace(",", "")                        # remove commas that separate thousands
    price = float(price[2:])                              # remove the sign of the currency
    return price
  
  # Dollar (always has decimal cases (i think) - thousands and decimals are both separated by commas)
  if price[0] == "$":
    price = replace_last_occurrence(price, ",", ".")      # replace comma that separates decimals by a dot
    price = price.replace(",", "")                        # remove commas that separate thousands
    price = float(price[1:])                              # remove the sign of the currency
    return price
  
  # Euro (decimal cases, if there are any, are separated by dots - thousands are separated by commas)
  if price[0] == "€":
    price = price.replace(",", "")                        # remove commas that separate thousands
    price = float(price[1:])                              # remove the sign of the currency
    return price                              
  
  return np.nan

def convert_prices_to_euros(row, currency_column, price_column):

  # Convert all prices to a base currency (euro)

  currency = row[currency_column]
  price = row[price_column]

  sol = 0.24
  dollar = 1/1.0993
  
  if currency in ["€"]:
    return price

  if currency in ["$"]:
    return dollar*price 
  
  if currency in ["S"]:
    return sol*price 


# Column with type of currency
df["currency"] = df_og["price"].str[0]

# Clean prices (remove commas, dots, and currency signs)
df["price_clean"] = df_og["price"].apply(clean_prices)

# Conver prices to euros
df["price_euro"] = df.apply(convert_prices_to_euros, args = ("currency", "price_clean"), axis = 1)

In [ ]:
df

,product,country,company,price,timestamp,offer_type,currency,price_clean,price_euro
0,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-22,unlocked,$,799.99,727.726735
1,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-21,unlocked,$,799.99,727.726735
2,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-20,unlocked,$,799.99,727.726735
3,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-19,unlocked,$,799.99,727.726735
4,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-18,unlocked,$,799.99,727.726735
...,...,...,...,...,...,...,...,...,...
602051,huawei Y9S 128GB,CL,claro,"$229,990",2020-06-21,unlocked,$,229.99,209.214955
602052,huawei Y9S 128GB,CO,Claro,"$1,168,900",2020-06-21,unlocked,$,1168.90,1063.313017
602053,huawei Y9S 128GB,CO,Tigo,"$1,079,900",2020-06-21,unlocked,$,1079.90,982.352406
602054,huawei Y9S 128GB,PE,Movistar,"S/1,519",2020-06-20,postpaid_portability,S,1519.00,364.560000


## Replace Company

In [ ]:
df = df.replace(to_replace="claro",value="Claro")

In [ ]:
df = df.replace(to_replace="movistar",value="Movistar")

## Analysis

In [ ]:
n_samples_per_product = df.groupby(['product']).size()
n_samples_per_product

product
alcatel 1 32GB             192
alcatel 1 8GB              476
alcatel 1S 32GB (2020)    2456
alcatel 1V 16GB (2019)      70
alcatel 2051               673
                          ... 
zte Blade V9               733
zte R340                   854
zte R341                   288
zte R550                   477
zte Z2371                  191
Length: 707, dtype: int64

In [ ]:
n_samples_per_product = df.groupby(['offer_type']).size()
n_samples_per_product

offer_type
postpaid_new_line        25470
postpaid_portability     11380
postpaid_renew             462
unlocked                564744
dtype: int64

In [ ]:
n_samples_per_product = df.groupby(['company']).size()
n_samples_per_product

company
Abcdin            29127
Avantel            2982
Bitel              3236
Claro             96348
Falabella         45516
Lider             12163
MEO               14134
Media Markt       14796
Movistar         159778
NOS               13695
Paris             28707
Radio Popular     19813
Ripley            32374
Tigo              17091
Vodafone          12834
Worten            16195
entel             57436
macOnline          2146
vtr                5533
wom               18152
dtype: int64

In [ ]:
n_samples_per_product = df.groupby(['country']).size()
n_samples_per_product

country
CL    354257
CO     78038
PE     78294
PT     91467
dtype: int64

In [ ]:
n_samples_per_product = df.groupby(['company', 'country', 'product']).size()
n_samples_per_product

company  country  product                                  
Abcdin   CL       alcatel 1S 32GB (2020)                       190
                  alcatel 1V 16GB (2019)                        70
                  alcatel 3X (2019) 64GB                       133
                  alcatel Tiger V Senior                        51
                  apple  iPhone 8 Plus 64GB Reacondicionado    393
                                                              ... 
wom      CO       vivo Y11s 32GB                               126
                  vivo Y20 64GB                                  6
                  xiaomi Redmi Note 8 128GB                     56
                  xiaomi Redmi Note 9 128GB                     56
                  zte Blade A3 Lite 32GB                       157
Length: 2920, dtype: int64

In [ ]:
df

,product,country,company,price,timestamp,offer_type,currency,price_clean,price_euro
0,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-22,unlocked,$,799.99,727.726735
1,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-21,unlocked,$,799.99,727.726735
2,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-20,unlocked,$,799.99,727.726735
3,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-19,unlocked,$,799.99,727.726735
4,xiaomi Pocophone 128GB,CL,Falabella,"$799,990",2020-07-18,unlocked,$,799.99,727.726735
...,...,...,...,...,...,...,...,...,...
602051,huawei Y9S 128GB,CL,Claro,"$229,990",2020-06-21,unlocked,$,229.99,209.214955
602052,huawei Y9S 128GB,CO,Claro,"$1,168,900",2020-06-21,unlocked,$,1168.90,1063.313017
602053,huawei Y9S 128GB,CO,Tigo,"$1,079,900",2020-06-21,unlocked,$,1079.90,982.352406
602054,huawei Y9S 128GB,PE,Movistar,"S/1,519",2020-06-20,postpaid_portability,S,1519.00,364.560000


In [ ]:
df["currency"]

0         $
1         $
2         $
3         $
4         $
         ..
602051    $
602052    $
602053    $
602054    S
602055    S
Name: currency, Length: 602056, dtype: object

In [ ]:
new_df=df.drop(columns=['price','currency','price_clean'])
new_df

,product,country,company,timestamp,offer_type,price_euro
0,xiaomi Pocophone 128GB,CL,Falabella,2020-07-22,unlocked,727.726735
1,xiaomi Pocophone 128GB,CL,Falabella,2020-07-21,unlocked,727.726735
2,xiaomi Pocophone 128GB,CL,Falabella,2020-07-20,unlocked,727.726735
3,xiaomi Pocophone 128GB,CL,Falabella,2020-07-19,unlocked,727.726735
4,xiaomi Pocophone 128GB,CL,Falabella,2020-07-18,unlocked,727.726735
...,...,...,...,...,...,...
602051,huawei Y9S 128GB,CL,Claro,2020-06-21,unlocked,209.214955
602052,huawei Y9S 128GB,CO,Claro,2020-06-21,unlocked,1063.313017
602053,huawei Y9S 128GB,CO,Tigo,2020-06-21,unlocked,982.352406
602054,huawei Y9S 128GB,PE,Movistar,2020-06-20,postpaid_portability,364.560000


In [ ]:
new_df.insert(loc=0, column='product_id', value=df.set_index(['company','country', 'product','offer_type']).index.factorize()[0]+1)
new_df

,product_id,product,country,company,timestamp,offer_type,price_euro
0,1,xiaomi Pocophone 128GB,CL,Falabella,2020-07-22,unlocked,727.726735
1,1,xiaomi Pocophone 128GB,CL,Falabella,2020-07-21,unlocked,727.726735
2,1,xiaomi Pocophone 128GB,CL,Falabella,2020-07-20,unlocked,727.726735
3,1,xiaomi Pocophone 128GB,CL,Falabella,2020-07-19,unlocked,727.726735
4,1,xiaomi Pocophone 128GB,CL,Falabella,2020-07-18,unlocked,727.726735
...,...,...,...,...,...,...,...
602051,3222,huawei Y9S 128GB,CL,Claro,2020-06-21,unlocked,209.214955
602052,3234,huawei Y9S 128GB,CO,Claro,2020-06-21,unlocked,1063.313017
602053,3228,huawei Y9S 128GB,CO,Tigo,2020-06-21,unlocked,982.352406
602054,3236,huawei Y9S 128GB,PE,Movistar,2020-06-20,postpaid_portability,364.560000


In [ ]:
print(max(new_df["product_id"]))

3236


In [ ]:
n_samples = new_df.groupby(by = ["product_id"]).size()
n_samples

product_id
1        81
2       281
3       119
4        56
5        23
       ... 
3232     48
3233    147
3234    204
3235     65
3236     32
Length: 3236, dtype: int64

In [ ]:
new_df2 = new_df.sort_values(by=['product_id','timestamp'], ascending=True)
new_df2

,product_id,product,country,company,timestamp,offer_type,price_euro
80,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-14,unlocked,727.726735
79,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-15,unlocked,727.726735
78,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-16,unlocked,727.726735
77,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-17,unlocked,727.726735
76,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-18,unlocked,727.726735
...,...,...,...,...,...,...,...
601733,3236,huawei Y9S 128GB,PE,Movistar,2020-07-17,postpaid_portability,364.560000
601720,3236,huawei Y9S 128GB,PE,Movistar,2020-07-18,postpaid_portability,364.560000
601707,3236,huawei Y9S 128GB,PE,Movistar,2020-07-19,postpaid_portability,364.560000
601693,3236,huawei Y9S 128GB,PE,Movistar,2020-07-20,postpaid_portability,364.560000


In [ ]:
##print(max(df["timestamp"]))
df_prod0 = new_df.loc[new_df['product']=='xiaomi Pocophone 128GB']
print(min(df_prod0["timestamp"]))
print(max(df_prod0["timestamp"]))


df_prod1 = df.loc[df['product']=='xiaomi Pocophone 128GB']
print(min(df_prod1["timestamp"]))
print(max(df_prod1["timestamp"]))
##print('Maximum value:', max_value)
##df_prod0 = df["product"]
##print(max(df["timestamp"][]))

2020-04-14 00:00:00
2020-07-22 00:00:00
2020-04-14 00:00:00
2020-07-22 00:00:00




*   Obter produtos com 1 única amostra



In [ ]:
df3=new_df2[new_df2.groupby(['product_id']).product_id.transform('count')<2]
df3

,product_id,product,country,company,timestamp,offer_type,price_euro
427,6,xiaomi Redmi Note 8 128GB,CO,Movistar,2021-06-01,postpaid_new_line,627.626672
6925,45,samsung Galaxy A31 128GB,PE,Claro,2020-11-08,postpaid_new_line,282.960000
7543,46,samsung Galaxy A31 128GB,PE,entel,2020-09-29,postpaid_portability,278.160000
13385,81,motorola Moto G8 Play 32GB,PE,Claro,2020-11-08,postpaid_new_line,138.960000
19951,116,samsung Galaxy A51 128GB,PE,entel,2021-03-08,postpaid_new_line,318.960000
...,...,...,...,...,...,...,...
593672,3197,samsung Galaxy A5 (2016) 16Gb,PT,Media Markt,2021-03-20,unlocked,249.000000
593673,3198,samsung Galaxy A5 (2017) 32GB,PT,Media Markt,2021-03-20,unlocked,259.000000
595665,3206,motorola Moto G7 Power 64GB,CL,Paris,2021-05-27,unlocked,109.151278
596812,3214,nokia 6.1 32GB,CL,Falabella,2020-08-21,unlocked,181.924861




*   Séires temporais com mais do que 15 amostras que será utilizado mais à frente



In [ ]:
df4=new_df2[new_df2.groupby(['product_id']).product_id.transform('count')>15]
df4

,product_id,product,country,company,timestamp,offer_type,price_euro
80,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-14,unlocked,727.726735
79,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-15,unlocked,727.726735
78,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-16,unlocked,727.726735
77,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-17,unlocked,727.726735
76,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-18,unlocked,727.726735
...,...,...,...,...,...,...,...
601733,3236,huawei Y9S 128GB,PE,Movistar,2020-07-17,postpaid_portability,364.560000
601720,3236,huawei Y9S 128GB,PE,Movistar,2020-07-18,postpaid_portability,364.560000
601707,3236,huawei Y9S 128GB,PE,Movistar,2020-07-19,postpaid_portability,364.560000
601693,3236,huawei Y9S 128GB,PE,Movistar,2020-07-20,postpaid_portability,364.560000


In [ ]:
df6=new_df2[new_df2.groupby(['product_id']).product_id.transform('count')<16]
df6

,product_id,product,country,company,timestamp,offer_type,price_euro
427,6,xiaomi Redmi Note 8 128GB,CO,Movistar,2021-06-01,postpaid_new_line,627.626672
878,11,xiaomi Redmi Note 8 128GB,CL,Lider,2020-07-16,unlocked,218.311653
876,11,xiaomi Redmi Note 8 128GB,CL,Lider,2020-07-17,unlocked,218.311653
874,11,xiaomi Redmi Note 8 128GB,CL,Lider,2020-07-18,unlocked,218.311653
872,11,xiaomi Redmi Note 8 128GB,CL,Lider,2020-07-19,unlocked,218.311653
...,...,...,...,...,...,...,...
595665,3206,motorola Moto G7 Power 64GB,CL,Paris,2021-05-27,unlocked,109.151278
596812,3214,nokia 6.1 32GB,CL,Falabella,2020-08-21,unlocked,181.924861
596982,3221,huawei Y9S 128GB,PE,Claro,2021-10-29,postpaid_new_line,153.360000
597944,3227,huawei Y9S 128GB,CO,Movistar,2021-05-31,postpaid_new_line,1182.497044


In [ ]:
df7 = df6.drop_duplicates(subset=['product_id'])
df7

,product_id,product,country,company,timestamp,offer_type,price_euro
427,6,xiaomi Redmi Note 8 128GB,CO,Movistar,2021-06-01,postpaid_new_line,627.626672
878,11,xiaomi Redmi Note 8 128GB,CL,Lider,2020-07-16,unlocked,218.311653
1205,17,blu Joy Adulto Mayor Sos,CL,Falabella,2020-04-14,unlocked,36.377695
6925,45,samsung Galaxy A31 128GB,PE,Claro,2020-11-08,postpaid_new_line,282.960000
7543,46,samsung Galaxy A31 128GB,PE,entel,2020-09-29,postpaid_portability,278.160000
...,...,...,...,...,...,...,...
595477,3204,huawei P30 128GB + P Smart 2019,CO,Claro,2020-03-31,unlocked,1673.701446
595665,3206,motorola Moto G7 Power 64GB,CL,Paris,2021-05-27,unlocked,109.151278
596812,3214,nokia 6.1 32GB,CL,Falabella,2020-08-21,unlocked,181.924861
596982,3221,huawei Y9S 128GB,PE,Claro,2021-10-29,postpaid_new_line,153.360000


In [ ]:
df8=new_df2[new_df2.groupby(['product_id']).product_id.transform('count')>1]
df8=df8[df8.groupby(['product_id']).product_id.transform('count')<16]
df8.insert(loc=0, column='ID', value=df8.set_index(['company','country', 'product', 'offer_type']).index.factorize()[0]+1)
df8= df8.drop_duplicates(subset=['ID'])
df8

,ID,product_id,product,country,company,timestamp,offer_type,price_euro
878,1,11,xiaomi Redmi Note 8 128GB,CL,Lider,2020-07-16,unlocked,218.311653
1205,2,17,blu Joy Adulto Mayor Sos,CL,Falabella,2020-04-14,unlocked,36.377695
8935,3,49,blu G9 64GB,CL,Falabella,2020-04-14,unlocked,209.214955
8937,4,50,nokia 3.1,CL,Paris,2021-03-24,unlocked,72.764486
10341,5,59,motorola One Action 128GB,CL,Paris,2020-08-01,unlocked,145.538070
...,...,...,...,...,...,...,...,...
586755,343,3165,samsung Galaxy S10+ 512GB,CL,Movistar,2020-08-18,postpaid_new_line,727.728555
593417,344,3189,apple iPhone 6S 64GB,CL,Abcdin,2021-08-30,unlocked,136.441372
595476,345,3203,huawei P30 128GB + P Smart 2019,CL,entel,2020-03-31,unlocked,398.326208
595477,346,3204,huawei P30 128GB + P Smart 2019,CO,Claro,2020-03-31,unlocked,1673.701446


In [ ]:
df4.insert(loc=0, column='ID', value=df4.set_index(['company','country', 'product', 'offer_type']).index.factorize()[0]+1)
df4

,ID,product_id,product,country,company,timestamp,offer_type,price_euro
80,1,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-14,unlocked,727.726735
79,1,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-15,unlocked,727.726735
78,1,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-16,unlocked,727.726735
77,1,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-17,unlocked,727.726735
76,1,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-18,unlocked,727.726735
...,...,...,...,...,...,...,...,...
601733,2762,3236,huawei Y9S 128GB,PE,Movistar,2020-07-17,postpaid_portability,364.560000
601720,2762,3236,huawei Y9S 128GB,PE,Movistar,2020-07-18,postpaid_portability,364.560000
601707,2762,3236,huawei Y9S 128GB,PE,Movistar,2020-07-19,postpaid_portability,364.560000
601693,2762,3236,huawei Y9S 128GB,PE,Movistar,2020-07-20,postpaid_portability,364.560000


In [ ]:
del df4['product_id']
df4

,ID,product,country,company,timestamp,offer_type,price_euro
80,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-14,unlocked,727.726735
79,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-15,unlocked,727.726735
78,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-16,unlocked,727.726735
77,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-17,unlocked,727.726735
76,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-18,unlocked,727.726735
...,...,...,...,...,...,...,...
601733,2762,huawei Y9S 128GB,PE,Movistar,2020-07-17,postpaid_portability,364.560000
601720,2762,huawei Y9S 128GB,PE,Movistar,2020-07-18,postpaid_portability,364.560000
601707,2762,huawei Y9S 128GB,PE,Movistar,2020-07-19,postpaid_portability,364.560000
601693,2762,huawei Y9S 128GB,PE,Movistar,2020-07-20,postpaid_portability,364.560000


In [ ]:
max(df4['ID'])

2762

In [ ]:
df5 = df4.drop_duplicates(subset=['ID'])
df5

,ID,product,country,company,timestamp,offer_type,price_euro
80,1,xiaomi Pocophone 128GB,CL,Falabella,2020-04-14,unlocked,727.726735
781,2,xiaomi Redmi Note 8 128GB,CO,Movistar,2021-01-22,unlocked,636.723369
432,3,xiaomi Redmi Note 8 128GB,CO,Tigo,2021-05-31,postpaid_new_line,600.291094
429,4,xiaomi Redmi Note 8 128GB,CO,wom,2021-06-01,unlocked,636.677886
337,5,xiaomi Redmi Note 8 128GB,CO,Claro,2021-07-03,unlocked,684.890385
...,...,...,...,...,...,...,...
599597,2758,huawei Y9S 128GB,CL,wom,2020-12-22,postpaid_new_line,212.862731
602055,2759,huawei Y9S 128GB,PE,Claro,2020-06-20,unlocked,273.360000
602052,2760,huawei Y9S 128GB,CO,Claro,2020-06-21,unlocked,1063.313017
602046,2761,huawei Y9S 128GB,PE,Falabella,2020-06-21,unlocked,335.760000


## Texto info

*   df -> matriz obtida da base de dados com a coluna do preço em euros 
*   new_df -> df com os product IDs e com o preço em euros
*   new_df2 -> mesma matriz mas ordenada
*   df3 -> produtos com apenas 1 amostra
*   df4 -> produtos com mais do que 15 amostras ordenado
*   df5 -> Apenas os produtos sem duplicados e ordenados por ID
*   df6 -> produtos com menos do que 16 amostras